# COVID-19 Predictions with ARIMA model

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import operator 
from pmdarima.arima import auto_arima
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
from datetime import date
import operator
from matplotlib import pyplot

In [ ]:
confirmed_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
coloco = ['Lat','Long']
confirmed_cases_global.drop(coloco, inplace=True, axis=1)
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Recovered']
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Diamond Princess']
confirmed_cases_global.head()


In [ ]:
#create a row for Canada
confirmed_cases_Canada=confirmed_cases_global[confirmed_cases_global['Country/Region']=='Canada']
confirmed_cases_Canada=confirmed_cases_Canada.groupby(['Country/Region']).sum().reset_index()
confirmed_cases_Canada.insert(0, 'Province/State', float('NaN'))
confirmed_cases_Canada

In [ ]:
confirmed_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region', 'Long_':'Long' }, inplace=True)
coloc = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key','Lat','Long']
confirmed_cases_US.drop(coloc, inplace=True, axis=1)
confirmed_cases_US.head()

In [ ]:
confirmed_cases_US=confirmed_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()

In [ ]:
deaths_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
colo = ['Lat','Long']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Recovered']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Diamond Princess']
deaths_cases_global.drop(colo, inplace=True, axis=1)
deaths_cases_global.head()

In [ ]:
deaths_cases_Canada=deaths_cases_global[deaths_cases_global['Country/Region']=='Canada']
deaths_cases_Canada=deaths_cases_Canada.groupby(['Country/Region']).sum().reset_index()
deaths_cases_Canada.insert(0, 'Province/State', float('NaN'))
deaths_cases_Canada

In [ ]:
deaths_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
deaths_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region','Long_':'Long' }, inplace=True)
col = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key', 'Population','Long','Lat']
deaths_cases_US.drop(col, inplace=True, axis=1)
deaths_cases_US.head()

In [ ]:
deaths_cases_US = deaths_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()

In [ ]:
confirmed_cases= pd.concat([confirmed_cases_global, confirmed_cases_US,confirmed_cases_Canada], axis=0, sort=False)

In [ ]:
deaths_cases = pd.concat([deaths_cases_global, deaths_cases_US,deaths_cases_Canada], axis=0, sort=False)

In [ ]:
recoveries_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
coloco = ['Lat','Long']
recoveries_cases.drop(coloco, inplace=True, axis=1)
recoveries_cases.head()

In [ ]:
colnames = confirmed_cases.columns.values

In [ ]:
colonames = recoveries_cases.columns.values

In [ ]:
confirmed = confirmed_cases.loc[:, colnames[4]:colnames[-1]]
deaths = deaths_cases.loc[:, colnames[4]:colnames[-1]]
recoveries = recoveries_cases.loc[:, colonames[4]:colonames[-1]]

In [ ]:
time_date = confirmed.columns.values

In [ ]:
all_days = np.array([i for i in range(len(time_date))]).reshape(-1, 1)

# Predictions for days ahead

In [ ]:
pred_days_ahead = 2
#pred_days_ahead = 7
#pred_days_ahead = 30
predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

predictions_ahead_plus_day = np.array([i for i in range(len(time_date)+pred_days_ahead+1)]).reshape(-1, 1)

In [ ]:
start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
predictions_ahead_dates = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day))]

In [ ]:
#confirmed_cases = confirmed_cases[~confirmed_cases['Country/Region'].isin(['Central African Republic','Chad'])]

#confirmed_cases = confirmed_cases[confirmed_cases['Country/Region'].isin(['Canada'])]
#confirmed_cases

In [ ]:
countries=confirmed_cases['Country/Region'].tolist()
province=confirmed_cases['Province/State'].tolist()

In [ ]:
prediction_models = dict()
prediction_models_confirmed = {'SVR': 0}
prediction_models_deaths = {'SVR': 0}
prediction_models_recoveries = {'SVR': 0}

In [ ]:
# Prediction Models

from sklearn import model_selection
from sklearn.svm import SVR
def SVM_regression(train_x, train_y, test_x, test_y, X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
   # c = [0.01, 0.1, 1]
   # gamma = [0.01, 0.1, 1]
   #epsilon = [0.01, 0.1, 1]
   #shrinking = [True, False]
    #degree = [3, 4, 5, 6]
  #grid_parameters = {'C': c, 'gamma' : gamma ,'epsilon': epsilon, 'shrinking' : shrinking, 'degree':degree} 
   #regressor = RandomizedSearchCV(SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1, C=0.1), grid_parameters, scoring='neg_mean_squared_error', 
    #                               cv=5)
    
    if (subpopulation == 'confirmed'):
         degree = 3
    elif (subpopulation == 'deaths'):
        degree = 4
    else :
        degree = 5


    regressor = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=degree, C=0.1)
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['SVR'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['SVR'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['SVR'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions) 
    

In [ ]:
from sklearn.ensemble import RandomForestRegressor
def RF_regression(train_x, train_y, test_x, test_y,X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    grid_parameters = {'n_estimators': n_estimators}
#    regressor = RandomizedSearchCV(RandomForestRegressor(), grid_parameters, scoring='neg_mean_squared_error', 
#                                   cv=3)
    
    regressor = RandomForestRegressor(n_estimators = 200, random_state = 42) 
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)


In [ ]:
# Gradient Boosted Regression Trees (GBRT) 
from sklearn.ensemble import GradientBoostingRegressor

def Grad_Boost_regression(train_x, train_y, test_x, test_y,X_test,subpopulation, X_train, Y_train):
   
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    loss = ['ls','lad','huber']
    grid_parameters = {'n_estimators': n_estimators}
  #  regressor = RandomizedSearchCV(GradientBoostingRegressor(learning_rate=0.1), grid_parameters, scoring='neg_mean_squared_error', 
 #                                  cv=3)
    
    regressor =  GradientBoostingRegressor(n_estimators=500, learning_rate=0.1,
                                            max_depth=1, random_state=0, loss='huber')
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)



In [ ]:
#check if a timeseries is stationary
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()     
    rolstd = timeseries.rolling(12).std()
    
    #Perform Dickey-Fuller test:
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput
    

In [ ]:
def pred_with_arima(timeseries,n_days_ahead):
    
    #find the optimal d 
    diff=timeseries-timeseries.shift(1)
    for d in range(1,10):
        if (test_stationarity(diff.dropna())['p-value']<0.1):
            break
        else: 
            d=d+1 
            diff=diff-diff.shift(1)
            
    #use the arima model to make predictions, use original timeseries with optimal d:
    model = pm.auto_arima(timeseries, start_p=0, start_q=0,
                             max_p=4, max_q=4,
                              seasonal=False,
                             d=d, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=False)
    #print(model)
    forecasts, conf_int =model.predict(n_days_ahead,return_conf_int=True ,alpha=0.05)
    
    return forecasts , conf_int  # returns a tuple, array of forecasts and all conf int

In [ ]:
def Arima_Analysis(region,pred_days_ahead):
   #create a dataframe for timeseries analysis
    date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
    df = pd.DataFrame(date_rng, columns=['date'])
    df['cases']=np.transpose(region) #this is a dataframe with dates and cases.
    df = df.set_index('date')
    df=df[df['cases'] != 0] #remove zeroes, they do not help for analysis. 
    if (len(df)>1):
        if(df['cases'][-1]-df['cases'][0]>5):
            pred_ts,conf_int=pred_with_arima(df['cases'],pred_days_ahead) #this is an array converted to a list
        else:
            pred_ts=[df['cases'][-1]+1]*pred_days_ahead
            conf_int=[np.array([df['cases'][-1],df['cases'][-1]+2])]*pred_days_ahead
    else:
        pred_ts=np.zeros(pred_days_ahead)
        conf_int=[np.zeros(pred_days_ahead)] * pred_days_ahead
    return pred_ts, conf_int

In [ ]:
import operator
def subpopulation_Analysis(cases,country,province, all_days,predictions_ahead,subpopulation, pred_days_ahead):
    
    pred  = list()
    if pd.isnull(province):
        region = cases.loc[(cases['Country/Region'] == country) & (pd.isnull(cases['Province/State']))]
    else :
        region = cases.loc[(cases['Country/Region'] == country) & (cases['Province/State'] == province)]
    print(region)
    
    reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
    region =  np.array(region.loc[:, colnames[4]:colnames[-1]]).reshape(-1, 1)
        
    #train_x, test_x, train_y, test_y = model_selection.train_test_split(all_days,region,test_size = 0.04,random_state = 42,shuffle=False)
    
    #pred_svm =  SVM_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_svm)
    
    #TODO: a for loop
    daily_increase_threshold1=np.absolute((reg[-1][-1]-reg[-1][-2])/reg[-1][-2])*100
    daily_increase_threshold2=np.absolute((reg[-1][-2]-reg[-1][-3])/reg[-1][-3])*100
    daily_increase_threshold3=np.absolute((reg[-1][-3]-reg[-1][-4])/reg[-1][-4])*100
    daily_increase_threshold4=np.absolute((reg[-1][-4]-reg[-1][-5])/reg[-1][-5])*100
    daily_increase_bound=1000
    daily_increase_threshold = max(daily_increase_threshold1,daily_increase_threshold2,daily_increase_threshold3,daily_increase_threshold4)
    
    if (daily_increase_threshold>daily_increase_bound):
        best_pred_ahead = 0  
        region = region
        best_pred = 0
        best_conf_int = np.zeros(2)
        reject_country = 1
        return(best_pred_ahead, region, best_pred, best_conf_int, reject_country)
     
    
    pred_arima, conf_int=Arima_Analysis(reg,pred_days_ahead)
    #pred.append(pred_svm)
    #pred_RF = RF_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_RF)

    
   # pred_gb =  Grad_Boost_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
   # pred.append(pred_gb)


    if (subpopulation == 'confirmed'): 
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_confirmed.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    elif (subpopulation == 'deaths'):
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_deaths.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    else :
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_recoveries.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
        

    best_pred_ahead = best_pred[-1]
    best_conf_int=conf_int[-1]
   
    best_conf_int[0]=max(reg[-1][-1], best_conf_int[0])
    if (best_conf_int[0]>best_conf_int[1]):
        best_conf_int[1]=2*reg[-1][-1]

    if(best_pred_ahead<reg[-1][-1]):
        best_pred_ahead= reg[-1][-1]+pred_days_ahead
    
    reject_country = 0
    
    return best_pred_ahead, region, best_pred, best_conf_int, reject_country

In [ ]:
pred_file = str(2)+"day_prediction_" +  date.today().strftime('%Y-%m-%d') + ".csv"
#pred_file = str(7)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + ".csv"
#pred_file = str(30)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file

In [ ]:
columns = ['Province/State','Country','Target/Date','N','low95N','high95N',
          'R','low95R','high95R','D','low95D','high95D','T','low95T','high95T',
           'M','low95M','high95M','C','low95C','high95C']

            

k = 0
for co in range(len(countries)):
    
    ignore_recoveries = 0    
    
    total_predictions = pd.DataFrame(columns = columns)
    
    k = k + 1
       
    best_pred_confirmed, region_con, best_pred_con, conf_int_con, reject_country = subpopulation_Analysis(confirmed_cases,countries[co],province[co],
                                                 all_days,predictions_ahead,"confirmed",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_conf')
        continue
        
    best_pred_deaths, region_d, best_pred_d, conf_int_d, reject_country = subpopulation_Analysis(deaths_cases,countries[co],province[co],
                                              all_days,predictions_ahead,"deaths",pred_days_ahead)
    if (reject_country==1):
        print('rejected_d')
        continue
        
    if ((countries[co] == 'US' or countries[co] == 'Canada' ) & (pd.notnull(province[co]))):
        ignore_recoveries=1   
    else:
        best_pred_recoveries, region_r, best_pred_r, conf_int_r, reject_country = subpopulation_Analysis(recoveries_cases,countries[co],province[co],
                                                  all_days,predictions_ahead,"recoveries",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_r')
        continue
    
    # Mortality rate
    if (conf_int_con[0]==0):
        mort_pred=0
        mort_pred_low=0
        mort_pred_up=0
    else:
        mort_pred  = best_pred_deaths/best_pred_confirmed
        mort_pred_low= conf_int_d[0]/conf_int_con[1]
        mort_pred_up=conf_int_d[1]/conf_int_con[0]
    
    #fix province
    province_name=''
    if (pd.notnull(province[co])) :
        province_name=province[co]
    
    if (ignore_recoveries==0):
        R_value=int(best_pred_recoveries) 
        low_R=int(conf_int_r[0])
        high_R=int(conf_int_r[1])
    else:
        R_value=''
        low_R=''
        high_R=''
        
    #Critical Cases:
    #Assume that 5% of all cases are critical and need hospitalization:
    serious_percentage=0.05
    critical=serious_percentage*best_pred_confirmed
    low_critical=serious_percentage*conf_int_con[0]
    high_critical=serious_percentage*conf_int_con[1]
    
    total_predictions = total_predictions.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed) ,
                                                'low95N':int(conf_int_con[0]),
                                                'high95N': int(conf_int_con[1]),
                                                'R': R_value ,
                                                'low95R' : low_R,
                                                'high95R' : high_R,
                                                'D': int(best_pred_deaths) ,
                                                'low95D' : int(conf_int_d[0]),
                                                'high95D': int(conf_int_d[1]),
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred,
                                                'low95M': mort_pred_low,
                                                'high95M': mort_pred_up ,
                                                'C': int(critical) ,
                                                'low95C':  int(low_critical) ,
                                                'high95C': int(high_critical) },ignore_index = True)
    
    if (k==1):
        header = True
    else:
        header = False
    
    total_predictions.to_csv(pred_file, sep=',',index = False,  mode='a', header = header)
    